## Import Libraries

In [12]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [13]:
import matplotlib.pyplot as plt
plt.ion()

In [14]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [15]:
from os import path, pardir

## Compile and program project

In [16]:
project_path = [path.curdir, "FFT-S"]
project_path

['.', 'FFT-S']

In [75]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include" -I"../../../../shared"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: FFT-S.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe" 

0

## Data Logging

In [62]:
live_plot = True

Create a figure for the plot.

In [63]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [64]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_fft"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment FFT",
}

Stop criteria to pass to the logger:

In [65]:
def stop_fn(logger_data):
    return len(logger_data.gpio) and all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [66]:
def stop_function(pin_values):
    return all(pin_values)

In [67]:
repetitions = 10

Perform the measurement.

In [76]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
        
charges_allocate = []
charges_fft = []
charges_free = []
times_allocate = []
times_fft = []
times_free = []


with DGILibExtra(**cd) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(10,stop_fn)
        charge2, time2 = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charge3, time3 = power_and_time_per_pulse(dgilib.data, 3, stop_function=stop_function)
        
        if len(charge2) != 2 or len(time2) != 2 or len(charge3) != 1 or len(time3) != 1:
            print(f"Parsing of measurement data failed, charge2:{charge2}, time2:{time2}, charge3:{charge3}, time3:{time3},")
            break
        charges_allocate.append(charge2[0])
        charges_fft.append(charge3[0])
        charges_free.append(charge2[1])
        times_allocate.append(time2[0])
        times_fft.append(time3[0])
        times_free.append(time2[1])
        dgilib.empty_data()

    charges = charges_allocate + charges_fft + charges_free 
    times = times_allocate + times_fft + times_free
    
    
    charges = [charge_allocate + charge_fft + charge_free for charge_allocate, charge_fft, charge_free in zip(charges_allocate, charges_fft, charges_free)]
    times = [time_allocate + time_fft + time_free for time_allocate, time_fft, time_free in zip(times_allocate, times_fft, times_free)]

    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [77]:
import json

config = {}
config["name"] = "FFT 128"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Allocate FFT", "Free FFT"], 3: ["FFT"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": ["Allocate FFT"], 
                                        "loop": ["FFT"],
                                        "exit": ["Free FFT"]}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

In [71]:
import pandas as pd

In [72]:
df = pd.DataFrame({'charges_allocate': charges_allocate, 'charges_fft':charges_fft, 'charges_free':charges_free, 'times_allocate':times_allocate, 'times_fft':times_fft, 'times_free':times_free})

In [73]:
df

,charges_allocate,charges_fft,charges_free,times_allocate,times_fft,times_free
0,0.000247,0.000056,2.191752e-08,0.504315,0.125931,0.000066
1,0.000247,0.000056,2.191593e-08,0.504191,0.125933,0.000066
2,0.000247,0.000056,2.272726e-08,0.504259,0.125929,0.000066
3,0.000247,0.000056,2.435103e-08,0.504263,0.125933,0.000066
4,0.000247,0.000056,2.191734e-08,0.504315,0.125927,0.000066
5,0.000248,0.000056,2.272800e-08,0.504248,0.125934,0.000066
6,0.000248,0.000056,2.272708e-08,0.504189,0.125932,0.000066
7,0.000247,0.000056,2.435103e-08,0.504193,0.125930,0.000066
8,0.000248,0.000056,2.435123e-08,0.504196,0.125868,0.000066
9,0.000247,0.000056,2.191823e-08,0.504116,0.125868,0.000066


In [74]:
df.describe()

,charges_allocate,charges_fft,charges_free,times_allocate,times_fft,times_free
count,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,100.000000,100.000000
mean,2.475056e-04,5.575679e-05,1.927000e-08,0.504148,0.125884,0.000057
std,1.019567e-07,3.316816e-08,7.871204e-09,0.000063,0.000032,0.000023
min,2.473052e-04,5.569098e-05,0.000000e+00,0.504041,0.125799,0.000000
25%,2.474312e-04,5.573273e-05,2.110406e-08,0.504109,0.125863,0.000066
50%,2.475041e-04,5.575854e-05,2.191717e-08,0.504126,0.125866,0.000066
75%,2.475833e-04,5.578179e-05,2.272910e-08,0.504185,0.125928,0.000066
max,2.477698e-04,5.584132e-05,2.516517e-08,0.504315,0.125934,0.000066
